In [1]:
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.models import Sequential
import time

# Load Dataset

In [2]:
# Download and load Shakespeare dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read the text file
with open(path_to_file, 'r') as file:
    text = file.read()

# Check the first 500 characters of the text
print(text[:500])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


# Preprocess Text
Convert the text into a numeric representation and prepare sequences of input and target text.

In [3]:
# Unique characters in the text
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

# Creating a mapping from unique characters to indices
char_to_index = {char: idx for idx, char in enumerate(vocab)}
index_to_char = np.array(vocab)

# Convert text to integers
text_as_int = np.array([char_to_index[c] for c in text])

print(f'Text as integers: {text_as_int[:20]}')

65 unique characters
Text as integers: [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56]


In [4]:
# Define sequence length and create training examples
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# Create the dataset by slicing text
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Create input-target sequences
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Function to split input and target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Display a sample input and target sequence
for input_example, target_example in dataset.take(1):
    print(f'Input: {repr("".join([index_to_char[i] for i in input_example.numpy()]))}')
    print(f'Target: {repr("".join([index_to_char[i] for i in target_example.numpy()]))}')

Input: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


# Prepare Training Dataset
Batch the dataset and shuffle it for better learning

In [5]:
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

# Create the batches
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Build the LSTM Model

In [6]:
# Length of the vocabulary in chars
embedding_dim = 256  # Dimension of the embedding layer
rnn_units = 1024     # Number of LSTM units

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])
    return model

# Build the model
model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


# Loss Function

In [8]:
# Define the loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

# Training the Model

In [9]:
# Number of epochs for training
EPOCHS = 30  # Increase this for better results

# Train the model
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/30
172/172 [==============================] - 520s 3s/step - loss: 2.6055
Epoch 2/30
172/172 [==============================] - 520s 3s/step - loss: 1.8947
Epoch 3/30
172/172 [==============================] - 523s 3s/step - loss: 1.6409
Epoch 4/30
172/172 [==============================] - 521s 3s/step - loss: 1.5063
Epoch 5/30
172/172 [==============================] - 513s 3s/step - loss: 1.4247
Epoch 6/30
172/172 [==============================] - 494s 3s/step - loss: 1.3682
Epoch 7/30
172/172 [==============================] - 501s 3s/step - loss: 1.3232
Epoch 8/30
172/172 [==============================] - 500s 3s/step - loss: 1.2858
Epoch 9/30
172/172 [==============================] - 490s 3s/step - loss: 1.2497
Epoch 10/30
172/172 [==============================] - 495s 3s/step - loss: 1.2160
Epoch 11/30
172/172 [==============================] - 492s 3s/step - loss: 1.1825
Epoch 12/30
172/172 [==============================] - 496s 3s/step - loss: 1.1471
Epoch 13/30
1

# Rebuild the Model for Inference
Rebuild the model with a batch size of 1 for text generation and load the trained weights.

In [11]:
# Define the necessary variables
vocab_size = len(vocab)  # Recalculate vocab_size in case it was lost

# Rebuild the model with batch size 1 for inference
model_for_inference = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# Load the trained weights into the new model
model_for_inference.set_weights(model.get_weights())


# Text Generation Function

In [12]:
def generate_text(model, start_string):
    # Number of characters to generate
    num_generate = 1000
    
    # Convert start string to numbers (vectorize)
    input_eval = [char_to_index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    # Empty string to store results
    text_generated = []
    
    # Temperature affects the creativity of the generated text
    temperature = 0.5  # Lower for more predictable text, higher for more diverse text
    
    # Reset model states
    model.reset_states()
    
    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        # Sample from the prediction distribution
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        
        # Add the predicted character to the input
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(index_to_char[predicted_id])
    
    return start_string + ''.join(text_generated)

In [13]:
# Generate text using the updated model
print(generate_text(model_for_inference, start_string="ROMEO: "))

ROMEO: I fain about your highness to my wife.

BAPTISTA:
Now, good my lord, let me hear him.

Shepherd:
None, sir; I know you fare you well, for I must wed;
But I will bring thee tidings, Call himself,
And spit it bleeding in thy country's breast,
And cannot live but to the senate-house.

KING EDWARD IV:
I would they would all for our state was banish'd:
And if I be not, hear'st me from the world,
To seek their fury on'd all in vain;
The sorrow that I was wont to come home.

BUCKINGHAM:
Faith, we may longer be a hundred throne.

GLOUCESTER:
So many give me leave to look on him.

NORTHUMBERLAND:
The northest issue like a treeban's face.

QUEEN ELIZABETH:
But how like you be well; where is he?

CATESBY:
Rescue, my Lord of Somerset, what would
Lear holy friar, that they have forget me
To banish them when I drink to thee again.
Therefore they thought it good or more your brother's love.

First Citizen:
We are all undone, we must be gone. We are not fit
with an aspect more than any other of